In [1]:
import pandas as pd
import numpy as np

from datetime import datetime

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelBinarizer, StandardScaler
from sklearn_pandas import DataFrameMapper

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from catboost import CatBoostClassifier

from sklearn.metrics import confusion_matrix, classification_report

from tensorflow.keras.models import Sequential
from tensorflow.keras import layers, Input, Model
from tensorflow.keras.layers import Dense, Dropout, PReLU, LeakyReLU

import keras
import tensorflow as tf

from keras.callbacks import EarlyStopping

import warnings
warnings.filterwarnings('ignore')

In [2]:
pd.set_option("max.columns", None)

In [3]:
df = pd.read_csv("both_home_away_ranks_with_games.csv")
del df["Unnamed: 0"], df["Unnamed: 0.1"]

In [4]:
df.head(10)

,start (et),visitor,pts_visitor,home,pts_home,ot,attendance,id,date,visitor_rank_assists,visitor_rank_blocks,visitor_rank_deffeciency,visitor_rank_drebounds,visitor_rank_fouls,visitor_rank_ft_pct,visitor_rankoeffeciency,visitor_rank_opp_assists,visitor_rank_opp_blocks,visitor_rank_opp_drebounds,visitor_rank_opp_fouls,visitor_rank_opp_orebounds,visitor_rank_opp_ptsfastbreak,visitor_rank_opp_ptsfrom2,visitor_rank_opp_ptsfrom3,visitor_rank_opp_ptsinpaint,visitor_rank_opp_ptspergame,visitor_rank_opp_steals,visitor_rank_opp_threepointpct,visitor_rank_opp_turnovers,visitor_rank_opp_twopointpct,visitor_rank_orebounds,visitor_rank_ptsfrom2,visitor_rank_ptsfrom3,visitor_rank_ptsfromfastbreak,visitor_rank_ptsinpaint,visitor_rank_scoring,visitor_rank_steals,visitor_rank_threeptpct,visitor_rank_turnovers,visitor_rank_twoptpct,home_rank_assists,home_rank_blocks,home_rank_deffeciency,home_rank_drebounds,home_rank_fouls,home_rank_ft_pct,home_rankoeffeciency,home_rank_opp_assists,home_rank_opp_blocks,home_rank_opp_drebounds,home_rank_opp_fouls,home_rank_opp_orebounds,home_rank_opp_ptsfastbreak,home_rank_opp_ptsfrom2,home_rank_opp_ptsfrom3,home_rank_opp_ptsinpaint,home_rank_opp_ptspergame,home_rank_opp_steals,home_rank_opp_threepointpct,home_rank_opp_turnovers,home_rank_opp_twopointpct,home_rank_orebounds,home_rank_ptsfrom2,home_rank_ptsfrom3,home_rank_ptsfromfastbreak,home_rank_ptsinpaint,home_rank_scoring,home_rank_steals,home_rank_threeptpct,home_rank_turnovers,home_rank_twoptpct
0,7:00p,Chicago Bulls,105,Charlotte Hornets,130,NaN,"15,136",53,2015-11-03,27,5,4,7,11,10,22,15,22,14,27,27,10,26,4,24,6,18,1,21,8,27,20,11,19,27,19,17,6,24,17,10,8,20,12,2,7,20,21,18,26,24,1,23,9,24,6,11,15,22,27,17,12,30,2,21,29,25,24,11,7,28
1,7:30p,Indiana Pacers,94,Detroit Pistons,82,NaN,"13,325",54,2015-11-03,25,21,22,29,20,13,27,10,3,13,7,24,8,25,7,26,20,28,26,5,18,25,28,15,7,30,26,4,8,27,25,24,29,5,10,3,30,17,5,19,19,5,2,9,17,6,20,4,12,8,15,22,1,24,14,25,17,22,28,18,9,30
2,7:30p,Atlanta Hawks,98,Miami Heat,92,NaN,"19,600",55,2015-11-03,7,23,15,16,5,11,15,25,23,18,30,29,18,3,29,2,10,13,21,9,2,26,13,16,4,15,21,5,21,11,5,8,10,17,14,7,1,4,13,2,10,26,10,3,6,15,18,7,5,4,28,24,29,19,7,13,6,13,20,1,8,10
3,8:00p,Orlando Magic,103,New Orleans Pelicans,94,NaN,"16,876",56,2015-11-03,13,1,13,17,22,24,25,9,29,20,25,30,12,24,13,16,24,16,5,4,16,5,8,17,9,3,9,10,22,25,24,18,27,29,27,14,19,18,27,13,25,6,15,25,22,30,23,30,22,27,24,28,19,11,13,16,11,10,23,14,16,19
4,8:30p,Toronto Raptors,102,Dallas Mavericks,91,NaN,"20,034",57,2015-11-03,15,30,6,9,28,18,6,19,9,3,4,4,28,2,10,7,8,29,14,12,4,8,18,4,28,12,4,3,2,26,18,4,28,12,1,24,20,16,4,24,29,14,11,22,4,20,3,9,4,6,29,12,21,14,8,14,20,15,29,17,1,16
5,10:00p,Memphis Grizzlies,103,Sacramento Kings,89,NaN,"17,317",58,2015-11-03,12,25,25,24,12,12,28,30,17,30,20,13,29,18,26,15,19,7,25,11,27,10,22,27,23,25,29,12,26,3,26,6,26,23,15,30,29,3,16,30,11,18,3,15,29,8,19,28,8,10,7,30,2,1,19,6,1,3,16,7,17,8
6,10:30p,Denver Nuggets,120,Los Angeles Lakers,109,NaN,"18,997",59,2015-11-03,19,12,16,21,17,23,29,20,27,17,16,22,21,8,19,17,14,25,16,3,15,6,26,9,24,19,27,14,12,20,29,26,4,27,19,23,5,10,29,14,24,10,12,27,30,14,30,29,3,13,30,29,17,25,3,11,26,7,26,23,13,15
7,7:00p,Boston Celtics,98,Indiana Pacers,100,NaN,"14,022",60,2015-11-04,3,18,7,27,24,3,25,9,16,28,7,20,2,6,17,14,15,29,21,1,10,13,19,22,5,8,14,1,26,23,23,24,21,17,29,15,9,28,5,3,11,8,24,3,23,7,25,13,27,22,2,14,24,27,15,11,30,27,2,12,27,22
8,7:00p,San Antonio Spurs,99,Washington Wizards,102,NaN,"17,721",61,2015-11-04,12,7,3,7,2,10,14,8,9,5,25,21,1,18,2,5,2,16,8,19,5,25,2,26,29,7,20,10,15,21,7,21,12,18,23,14,14,13,27,7,7,1,16,11,27,8,13,23,21,12,7,24,10,25,12,2,29,8,12,9,29,13
9,8:00p,Brooklyn Nets,87,Atlanta Hawks,101,NaN,"14,044",62,2015-11-04,23,24,28,28,6,23,26,24,4,12,21,8,8,11,28,15,22,11,29,16,27,26,4,29,28,24,29,27,28,10,11,5,26,12,16,3,12,16,15,24,20,27,29,17,3,26,6,8,17,16,4,7,22,12,13,6,10,22,5,20,11,8


In [5]:
df["home_team_won"] = 0
z = (df["pts_home"] > df["pts_visitor"])
df["home_team_won"][z] = 1

In [6]:
df["home_team_won"].value_counts()

1    3528
0    2583
Name: home_team_won, dtype: int64

In [7]:
3528 / (3528+2583)

0.5773195876288659

In [8]:
del df["date"], df["visitor"], df["pts_visitor"], df["home"], df["pts_home"], df["ot"], df["attendance"], df["id"]

In [9]:
df.head(1)

,start (et),visitor_rank_assists,visitor_rank_blocks,visitor_rank_deffeciency,visitor_rank_drebounds,visitor_rank_fouls,visitor_rank_ft_pct,visitor_rankoeffeciency,visitor_rank_opp_assists,visitor_rank_opp_blocks,visitor_rank_opp_drebounds,visitor_rank_opp_fouls,visitor_rank_opp_orebounds,visitor_rank_opp_ptsfastbreak,visitor_rank_opp_ptsfrom2,visitor_rank_opp_ptsfrom3,visitor_rank_opp_ptsinpaint,visitor_rank_opp_ptspergame,visitor_rank_opp_steals,visitor_rank_opp_threepointpct,visitor_rank_opp_turnovers,visitor_rank_opp_twopointpct,visitor_rank_orebounds,visitor_rank_ptsfrom2,visitor_rank_ptsfrom3,visitor_rank_ptsfromfastbreak,visitor_rank_ptsinpaint,visitor_rank_scoring,visitor_rank_steals,visitor_rank_threeptpct,visitor_rank_turnovers,visitor_rank_twoptpct,home_rank_assists,home_rank_blocks,home_rank_deffeciency,home_rank_drebounds,home_rank_fouls,home_rank_ft_pct,home_rankoeffeciency,home_rank_opp_assists,home_rank_opp_blocks,home_rank_opp_drebounds,home_rank_opp_fouls,home_rank_opp_orebounds,home_rank_opp_ptsfastbreak,home_rank_opp_ptsfrom2,home_rank_opp_ptsfrom3,home_rank_opp_ptsinpaint,home_rank_opp_ptspergame,home_rank_opp_steals,home_rank_opp_threepointpct,home_rank_opp_turnovers,home_rank_opp_twopointpct,home_rank_orebounds,home_rank_ptsfrom2,home_rank_ptsfrom3,home_rank_ptsfromfastbreak,home_rank_ptsinpaint,home_rank_scoring,home_rank_steals,home_rank_threeptpct,home_rank_turnovers,home_rank_twoptpct,home_team_won
0,7:00p,27,5,4,7,11,10,22,15,22,14,27,27,10,26,4,24,6,18,1,21,8,27,20,11,19,27,19,17,6,24,17,10,8,20,12,2,7,20,21,18,26,24,1,23,9,24,6,11,15,22,27,17,12,30,2,21,29,25,24,11,7,28,1


In [10]:
df["start (et)"] = df["start (et)"].str.replace("p", "")

In [11]:
df["start (et)"] = df["start (et)"].astype(str)

In [12]:
target = "home_team_won"
X = df.drop(target, axis=1)
y = df[target]

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=13)

In [14]:
mapper = DataFrameMapper([
    ('start (et)', LabelBinarizer()), 
    (['visitor_rank_assists'], StandardScaler()), 
    (['visitor_rank_blocks'], StandardScaler()),
    (['visitor_rank_deffeciency'], StandardScaler()), 
    (['visitor_rank_drebounds'], StandardScaler()),
    (['visitor_rank_fouls'], StandardScaler()), 
    (['visitor_rank_ft_pct'], StandardScaler()), 
    (['visitor_rankoeffeciency'], StandardScaler()),
    (['visitor_rank_opp_assists'], StandardScaler()), 
    (['visitor_rank_opp_blocks'], StandardScaler()),
    (['visitor_rank_opp_drebounds'], StandardScaler()), 
    (['visitor_rank_opp_fouls'], StandardScaler()),
    (['visitor_rank_opp_orebounds'], StandardScaler()), 
    (['visitor_rank_opp_ptsfastbreak'], StandardScaler()),
    (['visitor_rank_opp_ptsfrom2'], StandardScaler()), 
    (['visitor_rank_opp_ptsfrom3'], StandardScaler()),
    (['visitor_rank_opp_ptsinpaint'], StandardScaler()), 
    (['visitor_rank_opp_ptspergame'], StandardScaler()),
    (['visitor_rank_opp_steals'], StandardScaler()), 
    (['visitor_rank_opp_threepointpct'], StandardScaler()),
    (['visitor_rank_opp_turnovers'], StandardScaler()), 
    (['visitor_rank_opp_twopointpct'], StandardScaler()),
    (['visitor_rank_orebounds'], StandardScaler()), 
    (['visitor_rank_ptsfrom2'], StandardScaler()),
    (['visitor_rank_ptsfrom3'], StandardScaler()), 
    (['visitor_rank_ptsfromfastbreak'], StandardScaler()),
    (['visitor_rank_ptsinpaint'], StandardScaler()), 
    (['visitor_rank_scoring'], StandardScaler()),
    (['visitor_rank_steals'], StandardScaler()), 
    (['visitor_rank_threeptpct'], StandardScaler()),
    (['visitor_rank_turnovers'], StandardScaler()), 
    (['visitor_rank_twoptpct'], StandardScaler()), 
    (['home_rank_assists'], StandardScaler()),
    (['home_rank_blocks'], StandardScaler()), 
    (['home_rank_deffeciency'], StandardScaler()), 
    (['home_rank_drebounds'], StandardScaler()),
    (['home_rank_fouls'], StandardScaler()), 
    (['home_rank_ft_pct'], StandardScaler()), 
    (['home_rankoeffeciency'], StandardScaler()),
    (['home_rank_opp_assists'], StandardScaler()), 
    (['home_rank_opp_blocks'], StandardScaler()),
    (['home_rank_opp_drebounds'], StandardScaler()), 
    (['home_rank_opp_fouls'], StandardScaler()),
    (['home_rank_opp_orebounds'], StandardScaler()), 
    (['home_rank_opp_ptsfastbreak'], StandardScaler()),
    (['home_rank_opp_ptsfrom2'], StandardScaler()), 
    (['home_rank_opp_ptsfrom3'], StandardScaler()),
    (['home_rank_opp_ptsinpaint'], StandardScaler()), 
    (['home_rank_opp_ptspergame'], StandardScaler()),
    (['home_rank_opp_steals'], StandardScaler()), 
    (['home_rank_opp_threepointpct'], StandardScaler()),
    (['home_rank_opp_turnovers'], StandardScaler()), 
    (['home_rank_opp_twopointpct'], StandardScaler()),
    (['home_rank_orebounds'], StandardScaler()), 
    (['home_rank_ptsfrom2'], StandardScaler()), 
    (['home_rank_ptsfrom3'], StandardScaler()),
    (['home_rank_ptsfromfastbreak'], StandardScaler()), 
    (['home_rank_ptsinpaint'], StandardScaler()),
    (['home_rank_scoring'], StandardScaler()), 
    (['home_rank_steals'], StandardScaler()), 
    (['home_rank_threeptpct'], StandardScaler()),
    (['home_rank_turnovers'], StandardScaler()), 
    (['home_rank_twoptpct'], StandardScaler()) 
],df_out=True)

In [15]:
Z_train = mapper.fit_transform(X_train)
Z_test = mapper.transform(X_test)

In [16]:
model = LogisticRegression(max_iter=300).fit(Z_train, y_train)
print("Log regress train score is " + str(model.score(Z_train, y_train)))
print("Log regress test score is " + str(model.score(Z_test, y_test)))

Log regress train score is 0.6639755618590443
Log regress test score is 0.650523560209424


In [17]:
model = DecisionTreeClassifier().fit(Z_train, y_train)
print("Decision Tree train score is " + str(model.score(Z_train, y_train)))
print("Decision Tree test score is " + str(model.score(Z_test, y_test)))

Decision Tree train score is 1.0
Decision Tree test score is 0.5647905759162304


In [18]:
model = RandomForestClassifier().fit(Z_train, y_train)
print("Random Forest train score is " + str(model.score(Z_train, y_train)))
print("Random Forest test score is " + str(model.score(Z_test, y_test)))

Random Forest train score is 1.0
Random Forest test score is 0.6446335078534031


In [19]:
model = CatBoostClassifier().fit(Z_train, y_train)
print("Cat Boost train score is " + str(model.score(Z_train, y_train)))
print("Cat Boost test score is " + str(model.score(Z_test, y_test)))

Learning rate set to 0.019735
0:	learn: 0.6908643	total: 60.2ms	remaining: 1m
1:	learn: 0.6881519	total: 63.5ms	remaining: 31.7s
2:	learn: 0.6857121	total: 66.5ms	remaining: 22.1s
3:	learn: 0.6839079	total: 69.5ms	remaining: 17.3s
4:	learn: 0.6818117	total: 72.4ms	remaining: 14.4s
5:	learn: 0.6799465	total: 75.4ms	remaining: 12.5s
6:	learn: 0.6780057	total: 78.4ms	remaining: 11.1s
7:	learn: 0.6758440	total: 81.4ms	remaining: 10.1s
8:	learn: 0.6741132	total: 84.3ms	remaining: 9.29s
9:	learn: 0.6722854	total: 87.3ms	remaining: 8.64s
10:	learn: 0.6706009	total: 93.5ms	remaining: 8.4s
11:	learn: 0.6687764	total: 99.1ms	remaining: 8.15s
12:	learn: 0.6671303	total: 104ms	remaining: 7.9s
13:	learn: 0.6654453	total: 111ms	remaining: 7.8s
14:	learn: 0.6640368	total: 116ms	remaining: 7.61s
15:	learn: 0.6624964	total: 121ms	remaining: 7.43s
16:	learn: 0.6609998	total: 127ms	remaining: 7.35s
17:	learn: 0.6595665	total: 130ms	remaining: 7.1s
18:	learn: 0.6582089	total: 135ms	remaining: 6.95s
19:	le

169:	learn: 0.5692670	total: 666ms	remaining: 3.25s
170:	learn: 0.5689627	total: 670ms	remaining: 3.25s
171:	learn: 0.5687245	total: 675ms	remaining: 3.25s
172:	learn: 0.5682567	total: 680ms	remaining: 3.25s
173:	learn: 0.5680150	total: 686ms	remaining: 3.25s
174:	learn: 0.5676818	total: 689ms	remaining: 3.25s
175:	learn: 0.5674027	total: 692ms	remaining: 3.24s
176:	learn: 0.5670888	total: 695ms	remaining: 3.23s
177:	learn: 0.5667450	total: 698ms	remaining: 3.22s
178:	learn: 0.5663724	total: 701ms	remaining: 3.21s
179:	learn: 0.5660265	total: 704ms	remaining: 3.21s
180:	learn: 0.5657503	total: 707ms	remaining: 3.2s
181:	learn: 0.5653650	total: 710ms	remaining: 3.19s
182:	learn: 0.5650705	total: 713ms	remaining: 3.18s
183:	learn: 0.5645912	total: 716ms	remaining: 3.17s
184:	learn: 0.5641472	total: 719ms	remaining: 3.17s
185:	learn: 0.5638927	total: 722ms	remaining: 3.16s
186:	learn: 0.5636619	total: 724ms	remaining: 3.15s
187:	learn: 0.5634134	total: 727ms	remaining: 3.14s
188:	learn: 0

344:	learn: 0.5183170	total: 1.24s	remaining: 2.35s
345:	learn: 0.5180321	total: 1.25s	remaining: 2.35s
346:	learn: 0.5176930	total: 1.25s	remaining: 2.35s
347:	learn: 0.5174670	total: 1.25s	remaining: 2.35s
348:	learn: 0.5172988	total: 1.26s	remaining: 2.35s
349:	learn: 0.5169691	total: 1.26s	remaining: 2.34s
350:	learn: 0.5166746	total: 1.26s	remaining: 2.34s
351:	learn: 0.5163857	total: 1.27s	remaining: 2.33s
352:	learn: 0.5161363	total: 1.27s	remaining: 2.33s
353:	learn: 0.5158725	total: 1.27s	remaining: 2.33s
354:	learn: 0.5155970	total: 1.28s	remaining: 2.32s
355:	learn: 0.5152674	total: 1.28s	remaining: 2.32s
356:	learn: 0.5149445	total: 1.28s	remaining: 2.31s
357:	learn: 0.5146670	total: 1.29s	remaining: 2.31s
358:	learn: 0.5144313	total: 1.29s	remaining: 2.3s
359:	learn: 0.5141375	total: 1.29s	remaining: 2.3s
360:	learn: 0.5138492	total: 1.29s	remaining: 2.29s
361:	learn: 0.5136475	total: 1.3s	remaining: 2.29s
362:	learn: 0.5133395	total: 1.3s	remaining: 2.28s
363:	learn: 0.51

516:	learn: 0.4701414	total: 1.82s	remaining: 1.7s
517:	learn: 0.4698628	total: 1.82s	remaining: 1.7s
518:	learn: 0.4695217	total: 1.83s	remaining: 1.69s
519:	learn: 0.4691473	total: 1.83s	remaining: 1.69s
520:	learn: 0.4687629	total: 1.84s	remaining: 1.69s
521:	learn: 0.4685043	total: 1.84s	remaining: 1.68s
522:	learn: 0.4681662	total: 1.84s	remaining: 1.68s
523:	learn: 0.4677970	total: 1.84s	remaining: 1.68s
524:	learn: 0.4675291	total: 1.85s	remaining: 1.67s
525:	learn: 0.4672792	total: 1.85s	remaining: 1.67s
526:	learn: 0.4668999	total: 1.85s	remaining: 1.67s
527:	learn: 0.4665663	total: 1.86s	remaining: 1.66s
528:	learn: 0.4664334	total: 1.86s	remaining: 1.66s
529:	learn: 0.4661119	total: 1.86s	remaining: 1.65s
530:	learn: 0.4658002	total: 1.87s	remaining: 1.65s
531:	learn: 0.4656301	total: 1.87s	remaining: 1.65s
532:	learn: 0.4652401	total: 1.87s	remaining: 1.64s
533:	learn: 0.4649067	total: 1.88s	remaining: 1.64s
534:	learn: 0.4645461	total: 1.88s	remaining: 1.63s
535:	learn: 0.

690:	learn: 0.4193436	total: 2.39s	remaining: 1.07s
691:	learn: 0.4191232	total: 2.4s	remaining: 1.07s
692:	learn: 0.4189163	total: 2.4s	remaining: 1.06s
693:	learn: 0.4187330	total: 2.4s	remaining: 1.06s
694:	learn: 0.4184047	total: 2.41s	remaining: 1.06s
695:	learn: 0.4181378	total: 2.41s	remaining: 1.05s
696:	learn: 0.4178471	total: 2.41s	remaining: 1.05s
697:	learn: 0.4175736	total: 2.42s	remaining: 1.05s
698:	learn: 0.4174147	total: 2.42s	remaining: 1.04s
699:	learn: 0.4171725	total: 2.42s	remaining: 1.04s
700:	learn: 0.4168974	total: 2.43s	remaining: 1.03s
701:	learn: 0.4166319	total: 2.43s	remaining: 1.03s
702:	learn: 0.4165349	total: 2.43s	remaining: 1.03s
703:	learn: 0.4162239	total: 2.44s	remaining: 1.02s
704:	learn: 0.4160178	total: 2.44s	remaining: 1.02s
705:	learn: 0.4158114	total: 2.44s	remaining: 1.02s
706:	learn: 0.4155067	total: 2.44s	remaining: 1.01s
707:	learn: 0.4153016	total: 2.45s	remaining: 1.01s
708:	learn: 0.4150115	total: 2.45s	remaining: 1.01s
709:	learn: 0.4

866:	learn: 0.3755516	total: 2.96s	remaining: 454ms
867:	learn: 0.3753526	total: 2.97s	remaining: 451ms
868:	learn: 0.3750829	total: 2.97s	remaining: 448ms
869:	learn: 0.3749280	total: 2.98s	remaining: 445ms
870:	learn: 0.3746821	total: 2.98s	remaining: 441ms
871:	learn: 0.3744174	total: 2.98s	remaining: 438ms
872:	learn: 0.3742187	total: 2.99s	remaining: 435ms
873:	learn: 0.3739512	total: 2.99s	remaining: 431ms
874:	learn: 0.3737320	total: 2.99s	remaining: 428ms
875:	learn: 0.3734114	total: 3s	remaining: 424ms
876:	learn: 0.3731402	total: 3s	remaining: 421ms
877:	learn: 0.3727847	total: 3s	remaining: 417ms
878:	learn: 0.3724634	total: 3s	remaining: 414ms
879:	learn: 0.3722456	total: 3.01s	remaining: 410ms
880:	learn: 0.3720461	total: 3.01s	remaining: 407ms
881:	learn: 0.3718378	total: 3.01s	remaining: 403ms
882:	learn: 0.3715989	total: 3.02s	remaining: 400ms
883:	learn: 0.3712933	total: 3.02s	remaining: 396ms
884:	learn: 0.3710820	total: 3.02s	remaining: 393ms
885:	learn: 0.3708190	to

In [20]:
y_pred = model.predict(Z_test)

In [21]:
confusion_matrix(y_test, y_pred)

array([[340, 331],
       [196, 661]])

In [22]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.63      0.51      0.56       671
           1       0.67      0.77      0.71       857

    accuracy                           0.66      1528
   macro avg       0.65      0.64      0.64      1528
weighted avg       0.65      0.66      0.65      1528



In [23]:
model = Sequential([
    Input(shape=(Z_train.shape[1],)),
    Dense(64, activation="elu"),
    Dense(32, activation="elu"),
    Dense(1, activation="elu")
])

model.compile(loss="binary_crossentropy", optimizer="Adam", metrics=["accuracy"])

early_stop = EarlyStopping(monitor="val_loss", patience=5, verbose=0, mode="min")

model.fit(Z_train, y_train,
            validation_data=(Z_test, y_test),
            epochs=200, batch_size=8,
            verbose=2, callbacks=[early_stop])

Epoch 1/200
573/573 - 2s - loss: 6.9492 - accuracy: 0.4453 - val_loss: 6.6623 - val_accuracy: 0.4830
Epoch 2/200
573/573 - 0s - loss: 5.7288 - accuracy: 0.5481 - val_loss: 5.4397 - val_accuracy: 0.5975
Epoch 3/200
573/573 - 0s - loss: 5.5352 - accuracy: 0.5894 - val_loss: 5.3349 - val_accuracy: 0.6139
Epoch 4/200
573/573 - 0s - loss: 5.3028 - accuracy: 0.6077 - val_loss: 5.2521 - val_accuracy: 0.6198
Epoch 5/200
573/573 - 0s - loss: 5.2099 - accuracy: 0.6158 - val_loss: 5.0755 - val_accuracy: 0.6126
Epoch 6/200
573/573 - 0s - loss: 5.3532 - accuracy: 0.6014 - val_loss: 5.6527 - val_accuracy: 0.5897
Epoch 7/200
573/573 - 0s - loss: 5.3591 - accuracy: 0.5948 - val_loss: 5.2116 - val_accuracy: 0.6093
Epoch 8/200
573/573 - 0s - loss: 5.2181 - accuracy: 0.6123 - val_loss: 5.1351 - val_accuracy: 0.6158
Epoch 9/200
573/573 - 0s - loss: 5.0399 - accuracy: 0.6197 - val_loss: 4.8448 - val_accuracy: 0.6355
Epoch 10/200
573/573 - 0s - loss: 5.4794 - accuracy: 0.6053 - val_loss: 5.8334 - val_accura

In [24]:
model = LogisticRegression(max_iter=300).fit(Z_train, y_train)
print("Log regress train score is " + str(model.score(Z_train, y_train)))
print("Log regress test score is " + str(model.score(Z_test, y_test)))

Log regress train score is 0.6639755618590443
Log regress test score is 0.650523560209424


In [25]:
df2 = pd.read_csv("daily_data/full_daily_ranks.csv")

In [26]:
df2.head()

,Unnamed: 0,team,rank_assists,rank_blocks,rank_deffeciency,rank_drebounds,rank_fouls,rank_ft_pct,rank_oeffeciency,rank_opp_assists,rank_opp_blocks,rank_opp_drebounds,rank_opp_fouls,rank_opp_orebounds,rank_opp_ptsfastbreak,rank_opp_ptsfrom2,rank_opp_ptsfrom3,rank_opp_ptsinpaint,rank_opp_ptspergame,rank_opp_steals,rank_opp_threepointpct,rank_opp_turnovers,rank_opp_twopointpct,rank_orebounds,rank_ptsfrom2,rank_ptsfrom3,rank_ptsfromfastbreak,rank_ptsinpaint,rank_steals,rank_scoring,rank_threeptpct,rank_turnovers,rank_twoptpct
0,0,Golden State,1,16,8,12,28,13,21,7,10,27,9,28,22,11,14,12,21,22,15,3,4,29,22,7,6,20,6,14,10,25,13
1,1,Denver,2,20,14,19,9,9,4,28,6,1,17,4,26,10,16,7,9,15,7,15,28,7,6,13,13,9,8,4,6,13,4
2,2,Chicago,3,26,19,7,11,7,17,11,17,10,28,1,9,28,4,25,18,29,6,26,26,21,10,17,15,7,28,19,15,29,8
3,3,Memphis,4,19,9,10,12,14,15,23,24,20,25,12,25,13,19,15,20,16,13,7,17,3,2,26,1,1,1,12,19,8,19
4,4,Charlotte,5,15,17,26,3,20,19,30,13,17,23,27,23,4,30,13,15,24,14,10,29,9,25,8,10,21,12,22,8,26,21


In [27]:
df2["team"].unique

<bound method Series.unique of 0     Golden State
1           Denver
2          Chicago
3          Memphis
4        Charlotte
5          Phoenix
6         Brooklyn
7          Indiana
8      New Orleans
9            Miami
10       Milwaukee
11      Sacramento
12       Minnesota
13      Washington
14     San Antonio
15       LA Lakers
16     LA Clippers
17         Toronto
18         Detroit
19       Cleveland
20         Atlanta
21            Utah
22    Philadelphia
23          Boston
24         Houston
25          Dallas
26       Okla City
27         Orlando
28        New York
29        Portland
Name: team, dtype: object>

In [35]:
df_away = df2[(df2["team"] == "Toronto")]
del df_away["Unnamed: 0"], df_away["team"]

i = "visitor"
df_away = df_away.rename(columns={'rank_assists': str(i)+"_rank_assists",
       'rank_blocks':str(i)+"_rank_blocks", 'rank_deffeciency': str(i)+ "_rank_deffeciency", 'rank_drebounds': str(i)+ "_rank_drebounds",
       'rank_fouls': str(i)+"_rank_fouls",'rank_ft_pct': str(i)+"_rank_ft_pct", 'rank_oeffeciency': str(i)+"_rankoeffeciency", 'rank_opp_assists': str(i)+"_rank_opp_assists",
       'rank_opp_blocks': str(i)+"_rank_opp_blocks", 'rank_opp_drebounds': str(i)+"_rank_opp_drebounds", 'rank_opp_fouls': str(i)+"_rank_opp_fouls",
       'rank_opp_orebounds':str(i)+"_rank_opp_orebounds", 'rank_opp_ptsfastbreak': str(i)+"_rank_opp_ptsfastbreak", 'rank_opp_ptsfrom2': str(i)+"_rank_opp_ptsfrom2",
       'rank_opp_ptsfrom3': str(i)+"_rank_opp_ptsfrom3", 'rank_opp_ptsinpaint': str(i)+"_rank_opp_ptsinpaint", 'rank_opp_ptspergame': str(i)+"_rank_opp_ptspergame",
       'rank_opp_steals':str(i)+"_rank_opp_steals", 'rank_opp_threepointpct': str(i)+"_rank_opp_threepointpct", 'rank_opp_turnovers': str(i)+"_rank_opp_turnovers",
       'rank_opp_twopointpct': str(i)+"_rank_opp_twopointpct", 'rank_orebounds': str(i)+"_rank_orebounds", 'rank_ptsfrom2': str(i)+"_rank_ptsfrom2",
       'rank_ptsfrom3': str(i)+"_rank_ptsfrom3", 'rank_ptsfromfastbreak':str(i)+"_rank_ptsfromfastbreak", 'rank_ptsinpaint': str(i)+"_rank_ptsinpaint",
       'rank_scoring': str(i)+"_rank_scoring", 'rank_steals':str(i)+"_rank_steals", 'rank_threeptpct':str(i)+"_rank_threeptpct", 'rank_turnovers': str(i)+"_rank_turnovers",
       'rank_twoptpct':str(i)+"_rank_twoptpct"})

df_away = df_away.reset_index()

df_home = df2[(df2["team"] == "Denver")]
del df_home["Unnamed: 0"], df_home["team"]

i = "home"
df_home = df_home.rename(columns={'rank_assists': str(i)+"_rank_assists",
       'rank_blocks':str(i)+"_rank_blocks", 'rank_deffeciency': str(i)+ "_rank_deffeciency", 'rank_drebounds': str(i)+ "_rank_drebounds",
       'rank_fouls': str(i)+"_rank_fouls",'rank_ft_pct': str(i)+"_rank_ft_pct", 'rank_oeffeciency': str(i)+"_rankoeffeciency", 'rank_opp_assists': str(i)+"_rank_opp_assists",
       'rank_opp_blocks': str(i)+"_rank_opp_blocks", 'rank_opp_drebounds': str(i)+"_rank_opp_drebounds", 'rank_opp_fouls': str(i)+"_rank_opp_fouls",
       'rank_opp_orebounds':str(i)+"_rank_opp_orebounds", 'rank_opp_ptsfastbreak': str(i)+"_rank_opp_ptsfastbreak", 'rank_opp_ptsfrom2': str(i)+"_rank_opp_ptsfrom2",
       'rank_opp_ptsfrom3': str(i)+"_rank_opp_ptsfrom3", 'rank_opp_ptsinpaint': str(i)+"_rank_opp_ptsinpaint", 'rank_opp_ptspergame': str(i)+"_rank_opp_ptspergame",
       'rank_opp_steals':str(i)+"_rank_opp_steals", 'rank_opp_threepointpct': str(i)+"_rank_opp_threepointpct", 'rank_opp_turnovers': str(i)+"_rank_opp_turnovers",
       'rank_opp_twopointpct': str(i)+"_rank_opp_twopointpct", 'rank_orebounds': str(i)+"_rank_orebounds", 'rank_ptsfrom2': str(i)+"_rank_ptsfrom2",
       'rank_ptsfrom3': str(i)+"_rank_ptsfrom3", 'rank_ptsfromfastbreak':str(i)+"_rank_ptsfromfastbreak", 'rank_ptsinpaint': str(i)+"_rank_ptsinpaint",
       'rank_scoring': str(i)+"_rank_scoring", 'rank_steals':str(i)+"_rank_steals", 'rank_threeptpct':str(i)+"_rank_threeptpct", 'rank_turnovers': str(i)+"_rank_turnovers",
       'rank_twoptpct':str(i)+"_rank_twoptpct"})

df_home = df_home.reset_index()

df_predict = pd.concat([df_away, df_home], axis=1)
del df_predict["index"]

df_predict["start (et)"] = "9:00"

Z_new = mapper.transform(df_predict)
model.predict(Z_new)

array([1])